### When More is Less: Incorporating Additional Datasets Can Hurt Performance By Introducing Spurious Correlations (Report)
Authors: Aditya Asthana, Krish Desai

In [1]:
%pip install torch torchvision pandas matplotlib torchxrayvision

You should consider upgrading via the '/Users/adityaasthana/uiuc-mcs/CS598_Final_Report/cs598env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


#### 1. Pseudo-Labeling with TorchXRayVision
We’ll load a pretrained DenseNet121 (CheXpert-trained), run inference on each image, and threshold the “Pneumonia” score at 0.5 to create binary labels.


In [2]:
# **Cell 2: Imports & Globals**
# --------------------------------------------------------------
import os
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

import torchxrayvision as xrv
import matplotlib.pyplot as plt

# Paths
IMAGE_DIR = Path("data_sources_small")            # your raw scans
MANIFEST_DIR = Path("outputs_small")           # where CSVs will go
MANIFEST_DIR.mkdir(parents=True, exist_ok=True)


/Users/adityaasthana/uiuc-mcs/CS598_Final_Report/cs598env/lib/python3.8/site-packages/torchxrayvision/utils.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
# **Revised Cell 3: Load Pretrained Model & Preprocessor (CPU, 1-channel)**
import torchxrayvision as xrv
import torch
from torchvision import transforms
from PIL import Image

# Load TorchXRayVision DenseNet-121 (grayscale input)
model = xrv.models.DenseNet(weights="densenet121-res224-chex")
model = model.eval()  # keep on CPU; no .cuda()

# Preprocessing: resize → center-crop → grayscale → to-tensor → normalize
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=1),     # ensure 1 channel :contentReference[oaicite:0]{index=0}
    transforms.ToTensor(),                           # yields shape [1,224,224]
    transforms.Normalize(mean=[0.485], std=[0.229]), # single-channel norms :contentReference[oaicite:1]{index=1}
])


In [4]:
# **Revised Cell 4: Generate Pseudo-Labels (1-channel inputs)**
import os
import pandas as pd
import torch

records = []
for fname in sorted(os.listdir(IMAGE_DIR)):
    path = IMAGE_DIR / fname
    img = Image.open(path).convert("L")                # open as grayscale :contentReference[oaicite:2]{index=2}
    tensor = preprocess(img).unsqueeze(0)              # shape [1,1,224,224]
    with torch.no_grad():
        scores = model(tensor).numpy().squeeze()       # outputs [21] pathology logits
    idx = model.pathologies.index("Pneumonia")
    prob = torch.sigmoid(torch.tensor(scores[idx])).item()
    label = int(prob >= 0.5)
    records.append({"image_path": str(path), "label": label})

df_labels = pd.DataFrame(records)
df_labels.to_csv(MANIFEST_DIR/"pseudo_labels.csv", index=False)
df_labels.head()


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


,image_path,label
0,data_sources_small/00000001_000.png,1
1,data_sources_small/00000001_001.png,1
2,data_sources_small/00000001_002.png,1
3,data_sources_small/00000002_000.png,1
4,data_sources_small/00000003_000.png,1


#### 2. Create Two Domains
- **Domain A:** your original images  
- **Domain B:** apply a consistent augmentation (brightness & blur) to simulate a second “dataset”  


In [5]:
# **Cell 5: Define Augmentation & Build Manifests**
# --------------------------------------------------------------
aug_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.5, contrast=0.5),
    transforms.GaussianBlur(5),
])

# prepare lists
manifests = {"A": [], "B": []}

for _, row in df_labels.iterrows():
    path, lbl = row["image_path"], row["label"]
    # Domain A record
    manifests["A"].append({"image_path": path, "label": lbl, "domain": 0})
    # Domain B: save augmented copy to a temp folder
    img = Image.open(path).convert("RGB")
    aug = aug_transform(img)
    save_path = IMAGE_DIR/"domainB"/os.path.basename(path)
    save_path.parent.mkdir(exist_ok=True)
    aug.save(save_path)
    manifests["B"].append({"image_path": str(save_path), "label": lbl, "domain": 1})

!pip install scikit-learn
# Split each into train/val/test (80/10/10)
from sklearn.model_selection import train_test_split
for dom, recs in manifests.items():
    df = pd.DataFrame(recs)
    train, temp = train_test_split(df, stratify=df.label, test_size=0.2, random_state=0)
    val, test = train_test_split(temp, stratify=temp.label, test_size=0.5, random_state=0)
    train.to_csv(MANIFEST_DIR/f"{dom}_train.csv", index=False)
    val.to_csv(MANIFEST_DIR/f"{dom}_val.csv", index=False)
    test.to_csv(MANIFEST_DIR/f"{dom}_test.csv", index=False)


You should consider upgrading via the '/Users/adityaasthana/uiuc-mcs/CS598_Final_Report/cs598env/bin/python -m pip install --upgrade pip' command.


#### 3. Dataset & DataLoader
Define a PyTorch `Dataset` that reads our CSVs and returns `(img, label, domain)`.


In [6]:
# **Cell 6: Dataset Class**
# --------------------------------------------------------------
class ChestDataset(Dataset):
    def __init__(self, manifest_csv, transform=None):
        self.df = pd.read_csv(manifest_csv)
        self.transform = transform or transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485], std=[0.229]),
        ])
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row.image_path).convert("RGB")
        img = self.transform(img)
        return img, row.label, row.domain

def make_loader(dom, split, bs=32, shuffle=True):
    path = MANIFEST_DIR/f"{dom}_{split}.csv"
    return DataLoader(ChestDataset(path), batch_size=bs, shuffle=shuffle)

#### 4. Model & ERM Training
We’ll sample equally from each domain per step (“infinite loader” logic).

In [7]:
# **Cell 7 (CPU only): Model Factory & Training Loop**
import torch.nn as nn
import torch.optim as optim

def make_model():
    m = models.densenet121(pretrained=False)
    m.classifier = nn.Linear(m.classifier.in_features, 1)
    return m  # CPU model

def train_erm(domains, max_steps=5000, lr=1e-4):
    loaders = {d: make_loader(d, "train") for d in domains}
    iters = {d: iter(loaders[d]) for d in domains}
    model = make_model()            # CPU
    opt = optim.Adam(model.parameters(), lr=lr)
    crit = nn.BCEWithLogitsLoss()

    for step in range(max_steps):
        for d in domains:
            try:
                imgs, labs, _ = next(iters[d])
            except StopIteration:
                iters[d] = iter(loaders[d])
                imgs, labs, _ = next(iters[d])
            # no .cuda()
            preds = model(imgs).squeeze()
            loss = crit(preds, labs.float())
            loss.backward()
            opt.step()
            opt.zero_grad()
        if (step+1) % 1000 == 0:
            print(f"Step {step+1}/{max_steps}")
    return model


#### 5. Worst-Group Accuracy
Compute min accuracy over the four subgroups (domain×label).


In [8]:
# **Cell 8 (CPU only): Evaluation Metric**
import torch

def worst_group_acc(model, domains):
    allP, allL, allD = [], [], []
    for d in domains:
        loader = make_loader(d, "test", bs=64, shuffle=False)
        for imgs, labs, dom in loader:
            with torch.no_grad():
                out = torch.sigmoid(model(imgs)).round()  # CPU
            allP.append(out)
            allL.append(labs)
            allD.append(dom)
    P = torch.cat(allP)
    L = torch.cat(allL)
    D = torch.cat(allD)
    accs = []
    for d_val in torch.unique(D):
        for c in [0,1]:
            mask = (D==d_val)&(L==c)
            accs.append((P[mask]==L[mask]).float().mean().item())
    return min(accs)


#### 6. Run Experiments & Plot
Compare single-domain (A) vs. multi-domain (A+B).

In [ ]:
# **Cell 9: Execute & Compare**
# --------------------------------------------------------------
# Single-domain
modelA = train_erm(["A"])
wgA = worst_group_acc(modelA, ["A"])
# Multi-domain
modelAB = train_erm(["A","B"])
wgAB = worst_group_acc(modelAB, ["A","B"])

print(f"Single-domain worst-group acc: {wgA:.3f}")
print(f"Multi-domain worst-group acc:   {wgAB:.3f}")
print(f"Δ (multi – single):             {wgAB - wgA:+.3f}")


In [ ]:
# **Cell 10: Visualization**
# --------------------------------------------------------------
plt.bar(["Single","Multi"], [wgA, wgAB], color=["C0","C1"])
plt.ylabel("Worst-Group Accuracy")
plt.title("When More Is Less: Single vs. Multi Domain")
plt.ylim(0,1)
plt.show()


---

**Conclusion:**  
If Δ < 0, you’ve demonstrated that adding Domain B hurts the worst-group accuracy—i.e., “more is less,” reproducing the core finding from Compton et al.

Feel free to tweak:
- `max_steps`, learning rate, batch sizes  
- Pseudo-label threshold or pathology choice  
- Augmentation strength for Domain B  

to see how robust the phenomenon is in your own data!


In [ ]:
# **Cell X: Metrics & Evaluation Utilities**
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

def evaluate_model(model, domains, split="test", batch_size=64):
    """
    Runs the model on all domains for the given split and
    returns a DataFrame with columns:
      - domain (int)
      - y_true  (0/1)
      - y_pred  (0/1)
      - y_score (float, sigmoid output)
    """
    records = []
    model.eval()
    with torch.no_grad():
        for d in domains:
            loader = make_loader(d, split, bs=batch_size, shuffle=False)
            for imgs, labels, dom in loader:
                imgs = imgs  # CPU / CUDA as configured
                logits = model(imgs).squeeze()
                probs  = torch.sigmoid(logits).cpu().numpy()
                preds  = (probs >= 0.5).astype(int)
                
                # record each example
                for i in range(len(probs)):
                    records.append({
                        "domain": int(dom[i].item()),
                        "y_true": int(labels[i].item()),
                        "y_pred": int(preds[i]),
                        "y_score": float(probs[i]),
                    })
    return pd.DataFrame(records)

def compute_group_accuracies(df):
    """
    Given the DataFrame from evaluate_model, returns:
      - group_acc: dict[(domain, class) -> accuracy]
      - worst_group_acc: float
      - overall_acc: float
      - aurocs: dict[domain or 'overall' -> AUROC]
    """
    group_acc = {}
    for (d, c), sub in df.groupby(["domain", "y_true"]):
        if len(sub) == 0:
            acc = np.nan
        else:
            acc = (sub.y_pred == sub.y_true).mean()
        group_acc[(d, c)] = acc

    # worst‐group is the minimum over valid groups
    worst_group_acc = min(v for v in group_acc.values() if not np.isnan(v))

    # overall accuracy
    overall_acc = (df.y_pred == df.y_true).mean()

    # AUROC per domain and overall
    aurocs = {}
    # overall
    try:
        aurocs["overall"] = roc_auc_score(df.y_true, df.y_score)
    except ValueError:
        aurocs["overall"] = np.nan

    # per-domain
    for d, sub in df.groupby("domain"):
        try:
            aurocs[d] = roc_auc_score(sub.y_true, sub.y_score)
        except ValueError:
            aurocs[d] = np.nan

    return group_acc, worst_group_acc, overall_acc, aurocs

def eval_and_report(model, domains, split="test"):
    """
    Convenience wrapper: evaluate + compute metrics + print summary.
    Returns a dict with all metrics and the raw DataFrame.
    """
    df = evaluate_model(model, domains, split=split)
    group_acc, worst, overall, aurocs = compute_group_accuracies(df)

    print(f"--- Evaluation on split='{split}' domains={domains} ---")
    print(f"Overall Accuracy : {overall:.3f}")
    print(f"Worst-Group Acc  : {worst:.3f}")
    print("Group Accuracies :")
    for (d,c), acc in sorted(group_acc.items()):
        print(f"  Domain {d}, Class {c}: {acc:.3f}")
    print("AUROC Scores     :")
    for k,v in aurocs.items():
        print(f"  {k!s:>7}: {v:.3f}")
    
    # return for further use
    return {
      "df": df,
      "group_acc": group_acc,
      "worst_group_acc": worst,
      "overall_acc": overall,
      "aurocs": aurocs
    }


In [ ]:
metricsA   = eval_and_report(modelA, ["A"])
metricsAB  = eval_and_report(modelAB, ["A","B"])

# To export sub‐group results to CSV:
metricsAB["df"].to_csv("eval_AB.csv", index=False)